<a href="https://colab.research.google.com/github/atloveu1226/CMIT2024/blob/main/script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some important packages that are necessary in this project:

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import linprog
import time

With some parameters setting intially:

In [3]:
b   = [3, 1, 2, 2, 2, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0]

Aeq = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

beq = np.array(b[:8])

c = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]



lb = np.zeros(15)

ub = np.ones(15)

We define a nerual network, which we feed a 15-dimensional vector and output a 15-dimensional vector.

In [23]:
obs_size = 15
n_actions = 15  #it has 15 different kind of actions
HIDDEN_SIZE = 120

net= nn.Sequential(
            nn.Linear(obs_size, HIDDEN_SIZE),
            nn.ReLU(),
            #nn.Sigmoid(),
            nn.Linear(HIDDEN_SIZE, n_actions)
        )

objective = nn.CrossEntropyLoss() #quite standard for classification tasks
optimizer = optim.Adam(params=net.parameters(), lr=0.001)

It also define a 'state' function, to numerically to compare the result when we change the coefficient vector $𝐜$. The result shows that whether the diagonal sum of corresponding binary matrix satisfies the given condition.

In [5]:
def state(x):

  Dia_ob = b[8:15]

  #result = np.zeros(len(Dia_ob))

  count = 0

  i = 0

  #Translate diagonal restriction into matrix

  M = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

  # Get the temporary variable

  tem = np.dot(M, x)

  for i in range(len(Dia_ob)):

    if tem[i] == Dia_ob[i]:

      count = count + 1

  #state[count - 1] = 1

  return count

In [6]:
print(state(c))

4


We also need to define 'action' function, exerting action on $𝐜$.

In [7]:
sm = nn.Softmax(dim=1) #Softmax converts the 15-dimensional output vector to a probability distribution
def select_action(c):

  c_tensor = torch.FloatTensor([c])

  act_probs_t = sm(net(c_tensor))

  #print(act_probs_t)

  act_probs = act_probs_t.data.numpy()[0]

  action = np.random.choice(len(act_probs), p=act_probs) #chooses randomly one of the 4 actions according to the probabilities returned by the net

  return action

Example for the last function:

In [9]:
print('action is', select_action(c))

action is 12


To simplify the code, we write a new function for 'linprog'.

In [10]:
def object(c_vector):

  result = linprog(
    c_vector,
    A_eq = Aeq,
    b_eq = beq,
    bounds = list(zip(lb, ub)),
    method = 'highs',
   )

  return result

In [11]:
current_sol = object(c).x

print(state(current_sol))

1


We then define the 'next_action' function:

In [12]:
def next_step(c_vector, action):

  flag = False

  reward = 0

  result = c_vector.copy()

  #action = select_action(c)

  #print(action)

  #print(c)

  result[action] = 1 - c_vector[action]

  result = np.array(result)

  #print(type(c_vector))

  #print(type(result))

  #para.x is an array, para.success return a bool value

  para1 = object(c_vector)

  para2 = object(result)

  if para1.success == True and para2.success == True:

    if state(para1.x) < state(para2.x):

      reward = (state(para2.x)) / 7

      flag = True

    else:

      reward = 0

      flag = False

  #if state(para2.x) == 7 or para2.success == False:

    #flag = True

  #else:

    #flag = False



  return [result, reward, flag]

In [13]:
print(c, state(c))

for i in range(10):

  action = select_action(c)

  result,_ ,_= next_step(c, action)

  print(next_step(c, action), action)

  print(state(result))

#print(type(c))



[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 4
[array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), 1.0, True] 4
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]), 0, False] 13
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]), 0.42857142857142855, True] 10
3
[array([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]), 0, False] 7
5
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]), 1.0, True] 14
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]), 1.0, True] 14
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]), 0, False] 9
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]), 0, False] 8
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]), 0.42857142857142855, True] 10
3
[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]), 0, False] 9
3


Preparation for training.

In [21]:
BATCH_SIZE = 100 #200

GAMMA = 0.9

PERCENTILE = 30 #30
REWARD_GOAL = 0.98

from collections import namedtuple  #more readable tuples
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

Do the training process:

In [24]:
start_time = time.time()

iter_no = 0
reward_mean = 0
full_batch = []
batch = []
episode_steps = []
episode_reward = 0.0
current_state = c.copy()

while reward_mean < REWARD_GOAL:
        action = select_action(current_state)
        #print(type(current_state))
        next_state = next_step(current_state, action)[0]
        reward = next_step(current_state, action)[1]
        episode_is_done = next_step(current_state, action)[2]
        #print(next_step(current_state, action))

        episode_steps.append(EpisodeStep(observation=current_state, action=action))
        episode_reward += reward

        #print(episode_steps)

        if episode_is_done: # Episode finished
            batch.append(Episode(reward=episode_reward, steps=episode_steps))


            #print(len(batch))

            next_state = c.copy()
            episode_steps = []
            episode_reward = 0.0

            if len(batch) == BATCH_SIZE: # New set of batches ready --> select "elite"

                #print("Batch full")
                #print(batch)
                #print("\n")
                #input("Press Enter to continue...")


                reward_mean = float(np.mean(list(map(lambda s: s.reward, batch)))) #compute mean reward (lambda is inline function)
                #print(reward_mean)
                elite_candidates= batch
                #elite_candidates= batch + full_batch
                returnG = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), elite_candidates))
                reward_bound = np.percentile(returnG, PERCENTILE) #lowest score that is greater than PERCENTILE% of scores in the data set
                                                                  #Keep the highest 100-PERCENTILE %
                #print("Batch finished", returnG, reward_bound)
                #input("Press Enter to continue...")

                train_obs = []
                train_act = []
                elite_batch = []

                for example, discounted_reward in zip(elite_candidates, returnG):
                        if discounted_reward > reward_bound:
                        #if discounted_reward >= reward_bound:
                              train_obs.extend(map(lambda step: step.observation, example.steps))
                              train_act.extend(map(lambda step: step.action, example.steps))
                              elite_batch.append(example)
                full_batch=elite_batch
                current_state=train_obs
                acts=train_act

                #print(current_state)
                #print(acts)
                #input("Press Enter to continue...")

                #Do the training
                if len(full_batch) != 0 : # just in case empty during an iteration
                  state_t = torch.FloatTensor(current_state) #batch of states: [[1.0,0,0,0,0,0,0,0,0,0],[1,...]]
                  acts_t = torch.LongTensor(acts) # batch of actions: [0,2,3,1,..]

                  #print(state_t)
                  #print(acts_t)
                  #input("Press Enter to continue...")
                  optimizer.zero_grad() #it is good practice to do this, initializing the gradient computations
                  action_scores_t = net(state_t)

                  #print(action_scores_t)
                  #input("Press Enter to continue...")

                  loss_t = objective(action_scores_t, acts_t)
                  loss_t.backward() #computes the gradients
                  optimizer.step() #updates the weights according to the gradients
                  print("%d: loss=%.3f, reward_mean=%.3f" % (iter_no, loss_t.item(), reward_mean))
                  iter_no += 1
                batch = [] #empty the batch
            current_state = next_state

print("--- %s seconds ---" % (time.time() - start_time))

0: loss=2.712, reward_mean=0.886
1: loss=2.706, reward_mean=0.897
2: loss=2.694, reward_mean=0.851
3: loss=2.689, reward_mean=0.874
4: loss=2.689, reward_mean=0.863
5: loss=2.686, reward_mean=0.897
6: loss=2.648, reward_mean=0.937
7: loss=2.663, reward_mean=0.874
8: loss=2.628, reward_mean=0.926
9: loss=2.612, reward_mean=0.909
10: loss=2.654, reward_mean=0.909
11: loss=2.552, reward_mean=0.926
12: loss=2.597, reward_mean=0.897
13: loss=2.618, reward_mean=0.886
14: loss=2.586, reward_mean=0.926
15: loss=2.498, reward_mean=0.937
16: loss=2.522, reward_mean=0.949
17: loss=2.496, reward_mean=0.931
18: loss=2.453, reward_mean=0.971
19: loss=2.434, reward_mean=0.943
20: loss=2.446, reward_mean=0.966
21: loss=2.228, reward_mean=0.966
22: loss=2.199, reward_mean=0.960
23: loss=2.383, reward_mean=0.954
24: loss=2.343, reward_mean=0.954
25: loss=2.155, reward_mean=0.977
26: loss=2.101, reward_mean=0.949
27: loss=2.321, reward_mean=0.966
28: loss=2.359, reward_mean=0.960
29: loss=2.283, reward_m

In [32]:
done=0
counter=0
MAXCOUNTER=15
state_initial = c.copy()

print("Step: "+str(counter))
print(state_initial)

while done !=1 and counter<MAXCOUNTER:
  state_t = torch.FloatTensor([state_initial])
  action_scores_t = net(state_t)
  act_probs_t = sm(action_scores_t)
  #print(act_probs_t)
  act_probs = act_probs_t.data.numpy()[0]
  #proposed_action=np.argmax(act_probs)
  #print(proposed_action)
  next_state = next_step(state_initial, proposed_action)[0]
  #reward = next_step(state_initial, proposed_action)[1]
  #done = next_step(state_initial, proposed_action)[2]
  #stat_e=next_state.tolist() #changes NumPy array to a Python list
  counter+=1
  print("Step: "+str(counter))
  print(next_state)
  print((np.array(object(next_state).x)).reshape(3, 5))
  state_initial = next_state


Step: 0
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Step: 1
[1 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
[[ 0.  1.  0.  1.  1.]
 [ 1. -0.  0.  0. -0.]
 [ 1.  1.  0.  0.  0.]]
Step: 2
[1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
[[ 1.  1.  0.  1. -0.]
 [-0.  1.  0.  0.  0.]
 [ 1. -0.  0.  0.  1.]]
Step: 3
[1 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
[[ 0.  1.  0.  1.  1.]
 [ 1. -0.  0.  0. -0.]
 [ 1.  1.  0.  0.  0.]]
Step: 4
[1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
[[ 1.  1.  0.  1. -0.]
 [-0.  1.  0.  0.  0.]
 [ 1. -0.  0.  0.  1.]]
Step: 5
[1 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
[[ 0.  1.  0.  1.  1.]
 [ 1. -0.  0.  0. -0.]
 [ 1.  1.  0.  0.  0.]]
Step: 6
[1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
[[ 1.  1.  0.  1. -0.]
 [-0.  1.  0.  0.  0.]
 [ 1. -0.  0.  0.  1.]]
Step: 7
[1 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
[[ 0.  1.  0.  1.  1.]
 [ 1. -0.  0.  0. -0.]
 [ 1.  1.  0.  0.  0.]]
Step: 8
[1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
[[ 1.  1.  0.  1. -0.]
 [-0.  1.  0.  0.  0.]
 [ 1. -0.  0.  0.  1.]]
Step: 9
[1 0 1 0 1 0 1 0 0 0 0 1 0 0 0]
[[ 0.  1.  0.  1.  1.]
 [ 

In [33]:
print((object(c).x).reshape(3,5))

[[ 1.  1.  0.  1. -0.]
 [-0.  1.  0.  0.  0.]
 [ 1. -0.  0.  0.  1.]]
